In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from matplotlib import font_manager, rc

In [40]:
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 출력 생략 방지 크기 조절절
pd.set_option('display.max_info_rows', 1000000)
pd.set_option('display.max_info_columns', 500)

In [42]:
import os
print(os.getcwd()) # 파일 경로 확인

c:\Users\user\Desktop\zerosweet


### Data Preprocessing & Convert to CSV

In [23]:
for i in range(1, 20):
    df = pd.read_csv(f'c:/Users/user/Desktop/gg/output_data/식품안전_{i}_{50000 * i}.csv', encoding = 'utf-8')
    df = df.dropna().reset_index(drop=True) # 원재료명이 비어있는 row 삭제
    df = df[df['원재료명'].notnull() & (df['원재료명'].str.count(',') > 4)].reset_index(drop=True) # 원재료가 5개 이하는 제거

    # 정규표현식 패턴: '액'을 포함하되, '액젓'은 제외 (negative lookahead)
    pattern = r'액(?!젓)'

    # '액' 포함 + '액젓'이 아닌 행을 제거
    df = df[~df['제품명'].str.contains(pattern, flags=re.IGNORECASE, na=False, regex=True)].reset_index(drop=True)
    
    # 제거할 키워드 목록
    keywords = ['효소', '베이스', '향료', '추출', '분말', '색소', '혼합', '복합']
    
    # '원재료명'에 '알킬'이 포함된 행 제거
    df = df[~df['원재료명'].str.contains('알킬', na=False)].reset_index(drop=True)

    # 정규표현식 패턴 만들기 (예: '효소|베이스|향료')
    pattern = '|'.join(keywords)

    # 제품명에 해당 키워드가 포함되지 않은 행만 남기기
    df = df[~df['제품명'].str.contains(pattern, na=False)].reset_index(drop=True)
    df.info()
    
    df.to_csv(f'cleaned_food_data_{i}.csv', index=False, encoding='utf-8-sig')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26868 entries, 0 to 26867
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     26868 non-null  object
 1   업체명     26868 non-null  object
 2   원재료명    26868 non-null  object
 3   허가번호    26868 non-null  int64 
 4   품목보고일자  26868 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23073 entries, 0 to 23072
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     23073 non-null  object
 1   업체명     23073 non-null  object
 2   원재료명    23073 non-null  object
 3   허가번호    23073 non-null  int64 
 4   품목보고일자  23073 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 901.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26274 entries, 0 to 26273
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  -----

## 생성된 엑셀 파일 합치기

In [25]:
import glob


file_list = sorted(glob.glob('cleaned_food_data_*.csv'))
df_list = []

for file in file_list:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file, encoding='cp949')
        except Exception as e:
            print(f"⚠️ {file} 열기 실패: {e}")
            continue
    df_list.append(df)

# 병합
merged_df = pd.concat(df_list, ignore_index=True)

# 저장
merged_df.to_csv('cleaned_food_data_merged.csv', index=False, encoding='utf-8-sig')

⚠️ cleaned_food_data_merged.csv 열기 실패: Error tokenizing data. C error: Expected 8 fields in line 3, saw 11



### 안전, 유의, 대체당 속성으로 원재료 분류

In [ ]:
df = pd.read_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_with_gi_non_duplicate(big_class).csv', encoding='utf-8-sig')

# 키워드 리스트 정의
alt_sugar_keywords_safe = ['에리스리톨', '말티톨', '수크랄로스', '스테비아', '알룰로스']
alt_sugar_keywords_caution = ['아스파탐', '사카린']
caution_keywords = ['소르빈산', '벤조산', '나트륨', '아질산', '글루타민산나트륨', '설탕', '액상과당', '물엿']

# 원재료명을 분리해서 각 카테고리별로 다시 넣어주는 함수
def categorize_ingredients(ingredients):
    parts = str(ingredients).split(',')
    alt_sugar_safe = [p.strip() for p in parts if any(k in p for k in alt_sugar_keywords_safe)]
    alt_sugar_caution = [p.strip() for p in parts if any(k in p for k in alt_sugar_keywords_caution)]
    caution   = [p.strip() for p in parts if any(k in p for k in caution_keywords)]
    safe      = [p.strip() for p in parts if p.strip() not in alt_sugar_safe + alt_sugar_caution + caution]
    return pd.Series({
        '대체당(안전)': ', '.join(alt_sugar_safe),
        '대체당(주의)': ', '.join(alt_sugar_caution),
        '주의':   ', '.join(caution),
        '안전':   ', '.join(safe)
    })

# 적용해서 새 컬럼 생성
df[['대체당(안전)', '대체당(주의)', '주의', '안전']] = df['원재료명'].apply(categorize_ingredients)

# 결과 확인
print(df[['원재료명', '대체당(안전)', '대체당(주의)', '주의', '안전']].head())

# 필요하면 다시 CSV로 저장
df.to_csv('c:/Users/user/Desktop/gg/cleaned_food_data_labeled.csv',
          index=False, encoding='utf-8-sig')

                                                원재료명 대체당(안전) 대체당(주의)  \
0  냉동연육, 전분, 밀가루, 정제염, 소르빈산칼륨, 글루코노-δ-락톤, L-글루타민산...                   
1  파래김, 찹쌀, 참깨, 콩기름(대두유), 천일염, 연근, 새우분말, 귀리분말, 다시...                   
2               밀가루, 정제수, 타피오카전분, 가시오갈피분말, 알카리제, 정제염                   
3                        정제수, 쌀, 국, 효모, 아스파탐, 아세설팜칼륨            아스파탐   
4                         계피, 황기, 대추, 생강, 마늘, 간장, 돌게                   

                       주의                                                 안전  
0  소르빈산칼륨, L-글루타민산나트륨, 설탕                      냉동연육, 전분, 밀가루, 정제염, 글루코노-δ-락톤  
1                          파래김, 찹쌀, 참깨, 콩기름(대두유), 천일염, 연근, 새우분말, 귀리분말, 다시...  
2                                       밀가루, 정제수, 타피오카전분, 가시오갈피분말, 알카리제, 정제염  
3                                                      정제수, 쌀, 국, 효모, 아세설팜칼륨  
4                                                 계피, 황기, 대추, 생강, 마늘, 간장, 돌게  


### 이전 소스 코드 

In [110]:
df = pd.read_csv('c:/Users/user/Desktop/gg/output_data/식품안전_10_500000.csv', encoding='utf-8')
df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,산딸기크림_,(주)카카오브라운,"버터, 잼, 과채가공품, 설탕, 식품첨가물혼합제제",20160080733,20210512
1,산딸기크림치즈떡,농업회사법인(주)소부당,"찹쌀, 식물성크림, 가공치즈, 가공유크림, 산딸기과육, 산딸기퓌레, 정제수, 기타가...",20150478431,20240812
2,산딸기크림치즈롤,나폴레옹베이커리,"빵류, 가공유크림, 가공치즈, 우유, 유크림, 설탕, 냉동라즈베리, 과.채가공품, ...",20180108876,20201007
3,산딸기크림치즈페스츄리용 휴면반죽(수출용),에스피엘주식회사,"밀가루, 정제수, 가공버터, 밀가루, 연성치즈, 기타잼류, 기타잼류, 설탕, 탈지분...",20040306418,20090331
4,산딸기타르트,(주)카파니씨,"생크림, 설탕, 버터, 과채가공품, 난황, 정제수, 난백, 과채음료, 밀가루, 젤라...",20020036115,20210517
...,...,...,...,...,...
49995,수박퓨레,주식회사 아리랑 농업회사법인,수박퓌레,19990611193,20230712
49996,수박퓨레,래미에프앤비,"수박열매, 설탕",19980364259,20180704
49997,수박퓨레시럽,(주)프레시코,"구연산삼나트륨, 제이인산칼슘, 홍국색소, 구연산, 펙틴, 변성전분, 과.채가공품, ...",19910464049,20220726
49998,수박필링(KKD),아그라나프루트코리아주식회사,"설탕, 변성전분, 과.채가공품, 향료, 아미드펙틴, 글루코노-δ-락톤, 구연산삼나트...",19910443010,20240522


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     50000 non-null  object
 1   업체명     50000 non-null  object
 2   원재료명    49991 non-null  object
 3   허가번호    50000 non-null  int64 
 4   품목보고일자  50000 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


In [112]:
# 원재료명이 비어있는 5개 row 삭제(drop)
df = df.dropna().reset_index(drop=True)
df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,산딸기크림_,(주)카카오브라운,"버터, 잼, 과채가공품, 설탕, 식품첨가물혼합제제",20160080733,20210512
1,산딸기크림치즈떡,농업회사법인(주)소부당,"찹쌀, 식물성크림, 가공치즈, 가공유크림, 산딸기과육, 산딸기퓌레, 정제수, 기타가...",20150478431,20240812
2,산딸기크림치즈롤,나폴레옹베이커리,"빵류, 가공유크림, 가공치즈, 우유, 유크림, 설탕, 냉동라즈베리, 과.채가공품, ...",20180108876,20201007
3,산딸기크림치즈페스츄리용 휴면반죽(수출용),에스피엘주식회사,"밀가루, 정제수, 가공버터, 밀가루, 연성치즈, 기타잼류, 기타잼류, 설탕, 탈지분...",20040306418,20090331
4,산딸기타르트,(주)카파니씨,"생크림, 설탕, 버터, 과채가공품, 난황, 정제수, 난백, 과채음료, 밀가루, 젤라...",20020036115,20210517
...,...,...,...,...,...
49986,수박퓨레,주식회사 아리랑 농업회사법인,수박퓌레,19990611193,20230712
49987,수박퓨레,래미에프앤비,"수박열매, 설탕",19980364259,20180704
49988,수박퓨레시럽,(주)프레시코,"구연산삼나트륨, 제이인산칼슘, 홍국색소, 구연산, 펙틴, 변성전분, 과.채가공품, ...",19910464049,20220726
49989,수박필링(KKD),아그라나프루트코리아주식회사,"설탕, 변성전분, 과.채가공품, 향료, 아미드펙틴, 글루코노-δ-락톤, 구연산삼나트...",19910443010,20240522


- 결측치 행 제거 완료
- 이후 원재료 5개 이하 품목 제거 ( 5개이하는 순수 제품이라 가공품이 아닐 것이라 가정함. )

In [113]:
df = df[df['원재료명'].notnull() & (df['원재료명'].str.count(',') > 4)].reset_index(drop=True)

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26620 entries, 0 to 26619
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     26620 non-null  object
 1   업체명     26620 non-null  object
 2   원재료명    26620 non-null  object
 3   허가번호    26620 non-null  int64 
 4   품목보고일자  26620 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.0+ MB


In [115]:
df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,산딸기크림치즈떡,농업회사법인(주)소부당,"찹쌀, 식물성크림, 가공치즈, 가공유크림, 산딸기과육, 산딸기퓌레, 정제수, 기타가...",20150478431,20240812
1,산딸기크림치즈롤,나폴레옹베이커리,"빵류, 가공유크림, 가공치즈, 우유, 유크림, 설탕, 냉동라즈베리, 과.채가공품, ...",20180108876,20201007
2,산딸기크림치즈페스츄리용 휴면반죽(수출용),에스피엘주식회사,"밀가루, 정제수, 가공버터, 밀가루, 연성치즈, 기타잼류, 기타잼류, 설탕, 탈지분...",20040306418,20090331
3,산딸기타르트,(주)카파니씨,"생크림, 설탕, 버터, 과채가공품, 난황, 정제수, 난백, 과채음료, 밀가루, 젤라...",20020036115,20210517
4,산딸기타르틴,메종드조에,"펙틴, 럼레진, 버터, 설탕, 라즈베리, 분당, 계란, 박력분, 아몬드분말, 우유,...",20150107547,20160226
...,...,...,...,...,...
26615,수박통통,훼미리식품주식회사,"혼합제제, 레시틴, 락색소, 바닐린, 정제수, 밀가루, 준초콜릿, 물엿, 설탕, 쇼...",19820481021,20170515
26616,수박통통젤리,(주)서주 2공장,"물엿, 설탕, 올리고당, 젤라틴, D-소비톨액, 기타가공품, 산도조절제, 구연산삼나...",20170484590,20201224
26617,수박포도향 AH-F-2503,(주)삼화에프앤에프,"식품첨가물혼합제제용액, 수크랄로스, 정제수, 주정, 프로필렌글리콜, 식품첨가물혼합제제용액",19990309094,20190811
26618,수박퓨레시럽,(주)프레시코,"구연산삼나트륨, 제이인산칼슘, 홍국색소, 구연산, 펙틴, 변성전분, 과.채가공품, ...",19910464049,20220726


- 제품명 안에 효소, 베이스, 향료 등과 같은 제품 제거

In [116]:
import re

# 정규표현식 패턴: '액'을 포함하되, '액젓'은 제외 (negative lookahead)
pattern = r'액(?!젓)'

# '액' 포함 + '액젓'이 아닌 행을 제거
df = df[~df['제품명'].str.contains(pattern, flags=re.IGNORECASE, na=False, regex=True)].reset_index(drop=True)

In [117]:
# 제거할 키워드 목록
keywords = ['효소', '베이스', '향료', '추출', '분말', '색소', '혼합', '복합']

# 정규표현식 패턴 만들기 (예: '효소|베이스|향료')
pattern = '|'.join(keywords)

# 제품명에 해당 키워드가 포함되지 않은 행만 남기기
df = df[~df['제품명'].str.contains(pattern, na=False)].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24738 entries, 0 to 24737
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     24738 non-null  object
 1   업체명     24738 non-null  object
 2   원재료명    24738 non-null  object
 3   허가번호    24738 non-null  int64 
 4   품목보고일자  24738 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 966.5+ KB


In [118]:
df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,산딸기크림치즈떡,농업회사법인(주)소부당,"찹쌀, 식물성크림, 가공치즈, 가공유크림, 산딸기과육, 산딸기퓌레, 정제수, 기타가...",20150478431,20240812
1,산딸기크림치즈롤,나폴레옹베이커리,"빵류, 가공유크림, 가공치즈, 우유, 유크림, 설탕, 냉동라즈베리, 과.채가공품, ...",20180108876,20201007
2,산딸기크림치즈페스츄리용 휴면반죽(수출용),에스피엘주식회사,"밀가루, 정제수, 가공버터, 밀가루, 연성치즈, 기타잼류, 기타잼류, 설탕, 탈지분...",20040306418,20090331
3,산딸기타르트,(주)카파니씨,"생크림, 설탕, 버터, 과채가공품, 난황, 정제수, 난백, 과채음료, 밀가루, 젤라...",20020036115,20210517
4,산딸기타르틴,메종드조에,"펙틴, 럼레진, 버터, 설탕, 라즈베리, 분당, 계란, 박력분, 아몬드분말, 우유,...",20150107547,20160226
...,...,...,...,...,...
24733,수박통통,훼미리식품주식회사,"혼합제제, 레시틴, 락색소, 바닐린, 정제수, 밀가루, 준초콜릿, 물엿, 설탕, 쇼...",19820481021,20170515
24734,수박통통젤리,(주)서주 2공장,"물엿, 설탕, 올리고당, 젤라틴, D-소비톨액, 기타가공품, 산도조절제, 구연산삼나...",20170484590,20201224
24735,수박포도향 AH-F-2503,(주)삼화에프앤에프,"식품첨가물혼합제제용액, 수크랄로스, 정제수, 주정, 프로필렌글리콜, 식품첨가물혼합제제용액",19990309094,20190811
24736,수박퓨레시럽,(주)프레시코,"구연산삼나트륨, 제이인산칼슘, 홍국색소, 구연산, 펙틴, 변성전분, 과.채가공품, ...",19910464049,20220726


In [ ]:
df.to_csv('cleaned_food_data_10.csv', index=False, encoding='utf-8-sig') # csv 파일로 저장

### __성분별 빈도수 순위__

In [11]:
from collections import Counter

In [4]:
df = pd.read_csv('retry_code.csv', encoding='utf-8')
df

,식품코드,제품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,원산지국코드,원산지국명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자,업체명,원재료명,허가번호,품목보고일자
0,P101-101000100-0066,소프트팝콘,P,가공식품,1,가공식품,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,해당없음,2,수집,2021-06-30,2025-03-27,주에스디씨,"해조칼슘, 천일염, 설탕, 고올레산해바라기유, 옥수수가루",20060515035,20160309
1,P101-101000100-0190,상큼한충주딸기팝콘,P,가공식품,1,가공식품,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,해당없음,2,수집,2023-12-22,2025-03-27,예그린,"옥수수, 딸기농축액, 딸기레진, 야자유, 천일염",20180442405,20230929
2,P101-101000100-0355,산야초뿌리뻥튀기,P,가공식품,1,가공식품,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,해당없음,2,수집,2024-04-09,2025-03-27,김경신무설탕뻥튀기,"현미, 쌀, 보리, 귀리, 옥수수, 들깨, 천일염, 둥굴레뿌리, 알리움후커리알뿌리,...",20150520157,20230824
3,P101-101000100-0453,소금우유맛팝콘,P,가공식품,1,가공식품,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,해당없음,2,수집,2024-08-21,2025-03-27,주바프,"레시틴, 태움용융소금, 복합조미식품, 설탕, 혼합식용유, 옥수수",20030368161,20240819
4,P101-101000100-0454,솔티카라멜맛팝콘,P,가공식품,1,가공식품,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,해당없음,2,수집,2024-10-24,2025-03-27,주제이앤이아산공장,"혼합식용유, 당류가공품, 옥수수, 재제소금(재제조소금), 혼합제제, 레시틴",20100461303,20241024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,P124-200020000-0912,붕어빵믹스,P,가공식품,1,가공식품,24,기타식품류,24200,기타가공품,...,NaN,해당없음,2,수집,2024-12-17,2025-03-27,사조동아원주부산공장,"제일인산칼슘, 산성피로인산나트륨, 탄산수소나트륨, 잔탄검, 포도당, 유청분말, 정제...",19950129282,20221128
81533,P124-200020000-0913,비오프로틴즈흑임자,P,가공식품,1,가공식품,24,기타식품류,24200,기타가공품,...,NaN,해당없음,2,수집,2024-12-17,2025-03-27,주스포츠바이오텍,"곡류가공품, 두류가공품, 식물성크림혼합분말, 곡류가공품, 난소화성말토덱스트린(고시형...",20240530065,20241212
81534,P124-200020000-0914,비타그란비타c팝핑톡톡샤인머스캣맛,P,가공식품,1,가공식품,24,기타식품류,24200,기타가공품,...,NaN,해당없음,2,수집,2025-01-06,2025-03-27,주에스엘에스,"청포도분말, 황산망간, DL-사과산, 구연산, 혼합제제, 스테아린산마그네슘, 효소처...",20010306231,20250103
81535,P124-200020000-0915,비타제틱메가비타c2000,P,가공식품,1,가공식품,24,기타식품류,24200,기타가공품,...,NaN,해당없음,2,수집,2025-01-08,2025-03-27,주파낙스네이처팜,"산화아연, 비타민C",20210363169,20250102


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81537 entries, 0 to 81536
Data columns (total 169 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    식품코드                                   81537 non-null  object 
 1    제품명                                    81537 non-null  object 
 2    데이터구분코드                                81537 non-null  object 
 3    데이터구분명                                 81537 non-null  object 
 4    식품기원코드                                 81537 non-null  int64  
 5    식품기원명                                  81537 non-null  object 
 6    식품대분류코드                                81537 non-null  int64  
 7    식품대분류명                                 81537 non-null  object 
 8    대표식품코드                                 81537 non-null  int64  
 9    대표식품명                                  81537 non-null  object 
 10   식품중분류코드                                81537 non-null  i

In [9]:
df.columns

Index(['식품코드', '제품명', '데이터구분코드', '데이터구분명', '식품기원코드', '식품기원명', '식품대분류코드',
       '식품대분류명', '대표식품코드', '대표식품명',
       ...
       '원산지국코드', '원산지국명', '데이터생성방법코드', '데이터생성방법명', '데이터생성일자', '데이터기준일자', '업체명',
       '원재료명', '허가번호', '품목보고일자'],
      dtype='object', length=169)

In [ ]:
# NaN 제거 및 문자열 나누기
ingredients_series = df['원재료명'].dropna().str.split(',')

# 분리된 리스트를 모두 펼쳐서 하나의 시리즈로
flat_ingredients = ingredients_series.explode().str.strip()

# 빈도수 계산
ingredient_counts = flat_ingredients.value_counts()

# 퍼센트 계산
ingredient_percent = (ingredient_counts / len(flat_ingredients) * 100).round(2)

# 순위 계산
ingredient_rank = ingredient_counts.rank(method='dense', ascending=False).astype(int)

# 데이터프레임으로 정리
ingredient_stats = pd.DataFrame({
    '빈도수': ingredient_counts,
    '퍼센트': ingredient_percent,
    '순위': ingredient_rank
})

# 결과 보기
print(ingredient_stats.head(10))

          빈도수   퍼센트  순위
원재료명                   
정제수     34762  4.23   1
설탕      25571  3.11   2
소스      21828  2.66   3
정제소금    21509  2.62   4
기타가공품   17210  2.09   5
밀가루     14592  1.78   6
혼합제제    14550  1.77   7
복합조미식품  14381  1.75   8
과.채가공품  10328  1.26   9
곡류가공품   10299  1.25  10


In [20]:
df_indredient = ingredient_stats
df_indredient.to_csv('ingredient_frequency.csv', encoding='utf-8-sig')

In [23]:
# 인덱스를 '원재료명'이라는 컬럼으로 포함하고 저장
df_indredient.reset_index().rename(columns={'index': '원재료명'}).to_csv('C:/Users/user/Desktop/gg/output_data/preprocessing_data/ingredient_frequency.csv', encoding='utf-8-sig', index=False)

# __데이터 병합__

In [13]:
from tqdm import tqdm

# clean_name 함수 정의: 텍스트 정제
def clean_name(name):
    if pd.isna(name):
        return ''
    name = name.lower()
    name = re.sub(r'[^\w가-힣]', '', name)  # 특수문자 제거
    name = re.sub(r'\s+', '', name)        # 공백 제거
    return name

# 기본 데이터 로드 및 정제
df_n = pd.read_excel('C:/Users/user/Desktop/gg/output_data/20250327_foodlist.xlsx', engine='openpyxl')
df_n.columns = df_n.columns.str.strip()  # 열 이름 공백 제거

# '허가번호' 컬럼이 존재하는지 확인하고 처리
if '허가번호' in df_n.columns:
    df_n['품목제조보고번호'] = df_n['허가번호'].apply(clean_name)
else:
    print("허가번호 컬럼이 없습니다. 대신 '품목제조보고번호'를 사용합니다.")
    if '품목제조보고번호' in df_n.columns:
        df_n['품목제조보고번호'] = df_n['품목제조보고번호'].apply(clean_name)
    else:
        print("품목제조보고번호 컬럼도 없습니다. 필요한 컬럼을 확인해주세요.")
        # 필요한 다른 컬럼을 지정하거나 종료
        raise ValueError("필요한 컬럼이 존재하지 않습니다. 코드 종료.")

df_n['제품명'] = df_n['제품명'].apply(clean_name)
df_n['제조사명'] = df_n['제조사명'].apply(clean_name)

# 결과 저장 경로
output_path = 'C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.03'
os.makedirs(output_path, exist_ok=True)

# 원재료 데이터 경로
ingredient_path = 'C:/Users/user/Desktop/gg/output_data'

# tqdm을 사용하여 1~19번 파일 처리
for i in tqdm(range(1, 20), desc="파일 처리 중"):
    file_name = f'식품안전_{i}_{50000 * i}.csv'
    file_path = os.path.join(ingredient_path, file_name)

    try:
        df_i = pd.read_csv(file_path, encoding='utf-8-sig')
    except Exception as e:
        print(f"[ERROR] 파일 {file_name} 읽기 실패: {e}")
        continue

    # 정제: '업체명', '제품명' 컬럼에 대해 clean_name 함수 적용
    df_i['업체명'] = df_i['업체명'].apply(clean_name)
    df_i['제품명'] = df_i['제품명'].apply(clean_name)

    # 제품명과 제조사명에 대해 포함 관계로 매칭
    merged = pd.merge(df_i, df_n, left_on=['제품명', '업체명'], right_on=['제품명', '제조사명'], how='inner')

    if not merged.empty:
        # 병합된 결과 파일 저장
        output_file = os.path.join(output_path, f'data_{i}_merged.csv')
        merged.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✔ data_{i}_merged.csv 저장됨 ({len(merged)}건)")
    else:
        print(f"⚠ data_{i}_merged.csv에 매칭된 결과 없음")

허가번호 컬럼이 없습니다. 대신 '품목제조보고번호'를 사용합니다.


파일 처리 중:   5%|▌         | 1/19 [00:00<00:09,  1.93it/s]

✔ data_1_merged.csv 저장됨 (4617건)


파일 처리 중:  11%|█         | 2/19 [00:00<00:08,  2.10it/s]

✔ data_2_merged.csv 저장됨 (3699건)


...

✔ data_17_merged.csv 저장됨 (4125건)


파일 처리 중:  95%|█████████▍| 18/19 [00:08<00:00,  2.10it/s]

✔ data_18_merged.csv 저장됨 (4423건)


파일 처리 중: 100%|██████████| 19/19 [00:09<00:00,  2.10it/s]

✔ data_19_merged.csv 저장됨 (5418건)


In [14]:
def merge_files(output_path):
    # 결과물 파일을 저장할 경로
    merged_file = os.path.join(output_path, 'final_merged_data.csv')

    # 병합할 파일들을 리스트로 준비
    file_list = [f'data_{i}_merged.csv' for i in range(1, 20)]
    
    # 빈 DataFrame 준비
    merged_df = pd.DataFrame()

    for file_name in file_list:
        file_path = os.path.join(output_path, file_name)
        
        if os.path.exists(file_path):
            # 파일 읽기
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            # 파일 내용 병합
            merged_df = pd.concat([merged_df, df], ignore_index=True)
            print(f"✔ {file_name} 파일 병합됨")
        else:
            print(f"⚠ {file_name} 파일이 존재하지 않음")

    # 최종 병합된 데이터를 하나의 CSV로 저장
    merged_df.to_csv(merged_file, index=False, encoding='utf-8-sig')
    print(f"✔ 최종 병합된 파일이 {merged_file}에 저장되었습니다.")

if __name__ == '__main__':
    # 결과 저장 폴더 경로
    output_path = 'C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.03'
    
    # 병합 작업 실행
    merge_files(output_path)

✔ data_1_merged.csv 파일 병합됨
✔ data_2_merged.csv 파일 병합됨
✔ data_3_merged.csv 파일 병합됨
✔ data_4_merged.csv 파일 병합됨
✔ data_5_merged.csv 파일 병합됨
✔ data_6_merged.csv 파일 병합됨
✔ data_7_merged.csv 파일 병합됨
✔ data_8_merged.csv 파일 병합됨


C:\Users\user\AppData\Local\Temp\ipykernel_12856\3885018362.py:16: DtypeWarning: Columns (158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')
C:\Users\user\AppData\Local\Temp\ipykernel_12856\3885018362.py:16: DtypeWarning: Columns (158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')


✔ data_9_merged.csv 파일 병합됨
✔ data_10_merged.csv 파일 병합됨
✔ data_11_merged.csv 파일 병합됨
✔ data_12_merged.csv 파일 병합됨


C:\Users\user\AppData\Local\Temp\ipykernel_12856\3885018362.py:16: DtypeWarning: Columns (158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')


✔ data_13_merged.csv 파일 병합됨
✔ data_14_merged.csv 파일 병합됨
✔ data_15_merged.csv 파일 병합됨
✔ data_16_merged.csv 파일 병합됨
✔ data_17_merged.csv 파일 병합됨
✔ data_18_merged.csv 파일 병합됨


C:\Users\user\AppData\Local\Temp\ipykernel_12856\3885018362.py:16: DtypeWarning: Columns (158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')
C:\Users\user\AppData\Local\Temp\ipykernel_12856\3885018362.py:16: DtypeWarning: Columns (158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')


✔ data_19_merged.csv 파일 병합됨
✔ 최종 병합된 파일이 C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.03\final_merged_data.csv에 저장되었습니다.


In [ ]:
import dask.dataframe as dd
import pandas as pd
import os
import re
from tqdm import tqdm

# clean_name 함수 정의
def clean_name(name):
    if pd.isna(name):
        return ''
    name = name.lower()
    name = re.sub(r'[^\w가-힣]', '', name)  # 특수문자 제거
    name = re.sub(r'\s+', '', name)        # 공백 제거
    return name

# 기본 데이터 로드 및 정제
df_n = pd.read_excel('C:/Users/user/Desktop/gg/output_data/20250327_foodlist.xlsx', engine='openpyxl')
df_n.columns = df_n.columns.str.strip()  # 컬럼 이름 공백 제거
df_n['제조사명'] = df_n['제조사명'].apply(clean_name)
df_n['제품명'] = df_n['제품명'].apply(clean_name)

# 저장 경로
output_path = 'C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.02'
os.makedirs(output_path, exist_ok=True)

# 원재료 데이터 폴더
ingredient_path = 'C:/Users/user/Desktop/gg/output_data'

# Dask를 사용한 병렬 처리
for i in tqdm(range(1, 20), desc="파일 처리 중"):
    file_name = f'식품안전_{i}_{50000 * i}.csv'
    file_path = os.path.join(ingredient_path, file_name)

    try:
        # Dask DataFrame 로드
        df_i = dd.read_csv(file_path, encoding='utf-8-sig')
    except Exception as e:
        print(f"[ERROR] 파일 {file_name} 읽기 실패: {e}")
        continue

    # 정제 (Dask에서 적용할 수 있도록 Dask DataFrame에 적용)
    df_i['업체명'] = df_i['업체명'].apply(clean_name, meta=('x', 'str'))
    df_i['제품명'] = df_i['제품명'].apply(clean_name, meta=('x', 'str'))

    # 제품명과 제조사명이 부분 일치하는 경우 병합
    # 병합을 Dask에서 처리하려면 Dask DataFrame을 사용해야 함
    merged = dd.merge(df_i[['제품명', '업체명']], df_n[['제품명', '제조사명']], 
                      how='inner', left_on=['제품명', '업체명'], right_on=['제품명', '제조사명'])

    # 필터링: 제품명과 제조사명이 부분 포함될 때만
    filtered = merged[merged['제품명_y'].str.contains(merged['제품명_x'], na=False) & 
                      merged['제조사명_y'].str.contains(merged['업체명'], na=False)]

    if not filtered.compute().empty:
        # 컬럼 정리
        filtered = filtered.rename(columns={
            '제품명_x': '제품명_원재료',
            '제품명_y': '제품명_기준',
            '제조사명_y': '제조사명_기준'
        })

        # 저장
        output_file = os.path.join(output_path, f'data_{i}_merged.csv')
        filtered.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✔ data_{i}_merged.csv 저장됨 ({len(filtered)}건)")
    else:
        print(f"⚠ data_{i}_merged.csv에 매칭된 결과 없음")

In [ ]:
# 병합 대상 폴더
folder_path = 'C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.02'

# 병합 결과 리스트
dfs = []

# 기준 열 리스트 (정렬 및 누락 방지용)
base_columns = None

print("\n🔄 병합 시작...")

for file_name in sorted(os.listdir(folder_path)):
    if file_name.endswith('.csv') and file_name.startswith('data_'):
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            if df.empty:
                print(f"⚠ {file_name} → 건너뜀 (빈 파일)")
                continue

            # 기준 열 세팅
            if base_columns is None:
                base_columns = df.columns.tolist()
            else:
                for col in base_columns:
                    if col not in df.columns:
                        df[col] = ''
                df = df[base_columns]

            dfs.append(df)
            print(f"✔ {file_name} 병합됨 (행 수: {len(df)})")
        except Exception as e:
            print(f"[ERROR] {file_name} 읽기 실패: {e}")

# 전체 병합
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)

    # 중복 제거 (모든 열 기준)
    before_dedup = len(merged_df)
    merged_df.drop_duplicates(inplace=True)
    after_dedup = len(merged_df)

    # 저장
    output_file = os.path.join(folder_path, 'all.csv')
    merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 병합 완료: {before_dedup}행 → {after_dedup}행 (중복 제거됨)")
    print(f"📁 저장 위치: {output_file}")
else:
    print("\n❌ 병합할 파일이 없습니다.")

# 병합된 데이터 정제 (Merged Data Refined)

In [5]:
df = pd.read_csv('C:/Users/user/Desktop/gg/output_data/preprocessing_data/Ver.02/final_merged_data.csv', encoding='utf-8-sig')
df

,제품명,업체명,원재료명,허가번호,품목보고일자,식품코드,데이터구분코드,데이터구분명,식품기원코드,식품기원명,...,제조사명,수입업체명,유통업체명,수입여부,원산지국코드,원산지국명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",20030308122,20050616,P101-101000100-0145,P,가공식품,1,가공식품,...,쌀로만식품,해당없음,해당없음,N,NaN,해당없음,2,수집,2021-06-30,2025-03-27
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-106000100-3332,P,가공식품,1,가공식품,...,주더베이커리랩제2공장,해당없음,해당없음,N,NaN,해당없음,2,수집,2024-07-29,2025-03-27
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-405000100-0001,P,가공식품,1,가공식품,...,주더베이커리랩제2공장,해당없음,해당없음,N,NaN,해당없음,2,수집,2023-12-22,2025-03-27
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",19960538784,20240628,P120-200020300-0012,P,가공식품,1,가공식품,...,감포전촌젓갈,해당없음,해당없음,N,NaN,해당없음,2,수집,2024-07-03,2025-03-27
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",19850311001,20230728,P123-207020300-1205,P,가공식품,1,가공식품,...,태원식품산업주,해당없음,해당없음,N,NaN,해당없음,2,수집,2023-12-22,2025-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",20080493074,20181114,P123-201020300-2999,P,가공식품,1,가공식품,...,농업회사법인주식회사한우물,해당없음,해당없음,N,NaN,해당없음,2,수집,2021-06-30,2025-03-27
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",20080493074,20181116,P123-201020300-3000,P,가공식품,1,가공식품,...,농업회사법인주식회사한우물,해당없음,해당없음,N,NaN,해당없음,2,수집,2021-06-30,2025-03-27
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",20080493074,20181109,P123-201020300-3001,P,가공식품,1,가공식품,...,농업회사법인주식회사한우물,해당없음,해당없음,N,NaN,해당없음,2,수집,2021-06-30,2025-03-27
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",20080493074,20181114,P123-201020300-3002,P,가공식품,1,가공식품,...,농업회사법인주식회사한우물,해당없음,해당없음,N,NaN,해당없음,2,수집,2021-06-30,2025-03-27


In [6]:
df.columns

Index(['제품명', '업체명', '원재료명', '허가번호', '품목보고일자', '식품코드', '데이터구분코드', '데이터구분명',
       '식품기원코드', '식품기원명',
       ...
       '제조사명', '수입업체명', '유통업체명', '수입여부', '원산지국코드', '원산지국명', '데이터생성방법코드',
       '데이터생성방법명', '데이터생성일자', '데이터기준일자'],
      dtype='object', length=169)

In [7]:
ingredient_df = df.copy() # 데이터프레임 COPY

In [8]:
ingredient_df.dropna(how='all', axis=1, inplace=True) # 값이 한 개도 없는 속성 제거
ingredient_df

,제품명,업체명,원재료명,허가번호,품목보고일자,식품코드,데이터구분코드,데이터구분명,식품기원코드,식품기원명,...,품목제조보고번호,제조사명,수입업체명,유통업체명,수입여부,원산지국명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",20030308122,20050616,P101-101000100-0145,P,가공식품,1,가공식품,...,200303081221,쌀로만식품,해당없음,해당없음,N,해당없음,2,수집,2021-06-30,2025-03-27
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-106000100-3332,P,가공식품,1,가공식품,...,20130355300909,주더베이커리랩제2공장,해당없음,해당없음,N,해당없음,2,수집,2024-07-29,2025-03-27
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-405000100-0001,P,가공식품,1,가공식품,...,20130355300939,주더베이커리랩제2공장,해당없음,해당없음,N,해당없음,2,수집,2023-12-22,2025-03-27
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",19960538784,20240628,P120-200020300-0012,P,가공식품,1,가공식품,...,199605387849,감포전촌젓갈,해당없음,해당없음,N,해당없음,2,수집,2024-07-03,2025-03-27
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",19850311001,20230728,P123-207020300-1205,P,가공식품,1,가공식품,...,1985031100113479,태원식품산업주,해당없음,해당없음,N,해당없음,2,수집,2023-12-22,2025-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",20080493074,20181114,P123-201020300-2999,P,가공식품,1,가공식품,...,20080493074492,농업회사법인주식회사한우물,해당없음,해당없음,N,해당없음,2,수집,2021-06-30,2025-03-27
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",20080493074,20181116,P123-201020300-3000,P,가공식품,1,가공식품,...,20080493074494,농업회사법인주식회사한우물,해당없음,해당없음,N,해당없음,2,수집,2021-06-30,2025-03-27
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",20080493074,20181109,P123-201020300-3001,P,가공식품,1,가공식품,...,20080493074490,농업회사법인주식회사한우물,해당없음,해당없음,N,해당없음,2,수집,2021-06-30,2025-03-27
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",20080493074,20181114,P123-201020300-3002,P,가공식품,1,가공식품,...,20080493074491,농업회사법인주식회사한우물,해당없음,해당없음,N,해당없음,2,수집,2021-06-30,2025-03-27


In [9]:
ingredient_df.nunique()

제품명          70229
업체명           7349
원재료명         68081
허가번호          7586
품목보고일자        5049
             ...  
원산지국명            1
데이터생성방법코드        2
데이터생성방법명         2
데이터생성일자        445
데이터기준일자          1
Length: 124, dtype: int64

In [10]:
# 고유값이 1개인 열 찾기
cols_to_drop = ingredient_df.nunique()[ingredient_df.nunique() == 1].index

# 해당 열들 제거
ingredient_df = ingredient_df.drop(columns=cols_to_drop)

In [11]:
ingredient_df

,제품명,업체명,원재료명,허가번호,품목보고일자,식품코드,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,출처코드,출처명,1회 섭취참고량,식품중량,품목제조보고번호,제조사명,유통업체명,데이터생성방법코드,데이터생성방법명,데이터생성일자
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",20030308122,20050616,P101-101000100-0145,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,3,식품의약품안전처,20g,300g,200303081221,쌀로만식품,해당없음,2,수집,2021-06-30
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-106000100-3332,1,과자류·빵류 또는 떡류,1106,일반과자,...,3,식품의약품안전처,30g,20g,20130355300909,주더베이커리랩제2공장,해당없음,2,수집,2024-07-29
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-405000100-0001,1,과자류·빵류 또는 떡류,1405,기타 빵,...,3,식품의약품안전처,70g,100g,20130355300939,주더베이커리랩제2공장,해당없음,2,수집,2023-12-22
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",19960538784,20240628,P120-200020300-0012,20,수산가공식품류,20200,젓갈/액젓,...,3,식품의약품안전처,-,100g,199605387849,감포전촌젓갈,해당없음,2,수집,2024-07-03
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",19850311001,20230728,P123-207020300-1205,23,즉석식품류,23207,국/탕류,...,3,식품의약품안전처,250ml(g),600g,1985031100113479,태원식품산업주,해당없음,2,수집,2023-12-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",20080493074,20181114,P123-201020300-2999,23,즉석식품류,23201,밥류,...,3,식품의약품안전처,210g,210g,20080493074492,농업회사법인주식회사한우물,해당없음,2,수집,2021-06-30
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",20080493074,20181116,P123-201020300-3000,23,즉석식품류,23201,밥류,...,3,식품의약품안전처,210g,210g,20080493074494,농업회사법인주식회사한우물,해당없음,2,수집,2021-06-30
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",20080493074,20181109,P123-201020300-3001,23,즉석식품류,23201,밥류,...,3,식품의약품안전처,210g,210g,20080493074490,농업회사법인주식회사한우물,해당없음,2,수집,2021-06-30
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",20080493074,20181114,P123-201020300-3002,23,즉석식품류,23201,밥류,...,3,식품의약품안전처,210g,210g,20080493074491,농업회사법인주식회사한우물,해당없음,2,수집,2021-06-30


In [12]:
ingredient_df.nunique()

제품명          70229
업체명           7349
원재료명         68081
허가번호          7586
품목보고일자        5049
             ...  
제조사명          7349
유통업체명          382
데이터생성방법코드        2
데이터생성방법명         2
데이터생성일자        445
Length: 94, dtype: int64

In [13]:
ingredient_df['데이터생성방법코드'].unique()

array([2, 1])

In [14]:
# 고유값이 2개인 열 찾기
cols_to_drop_2 = ingredient_df.nunique()[ingredient_df.nunique() == 2].index

# 해당 열들 제거
ingredient_df = ingredient_df.drop(columns=cols_to_drop_2)

ingredient_df

,제품명,업체명,원재료명,허가번호,품목보고일자,식품코드,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),1회 섭취참고량,식품중량,품목제조보고번호,제조사명,유통업체명,데이터생성일자
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",20030308122,20050616,P101-101000100-0145,1,과자류·빵류 또는 떡류,1101,강냉이/팝콘,...,NaN,NaN,NaN,NaN,20g,300g,200303081221,쌀로만식품,해당없음,2021-06-30
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-106000100-3332,1,과자류·빵류 또는 떡류,1106,일반과자,...,NaN,NaN,NaN,NaN,30g,20g,20130355300909,주더베이커리랩제2공장,해당없음,2024-07-29
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",20130355300,20230828,P101-405000100-0001,1,과자류·빵류 또는 떡류,1405,기타 빵,...,NaN,NaN,NaN,NaN,70g,100g,20130355300939,주더베이커리랩제2공장,해당없음,2023-12-22
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",19960538784,20240628,P120-200020300-0012,20,수산가공식품류,20200,젓갈/액젓,...,NaN,NaN,NaN,NaN,-,100g,199605387849,감포전촌젓갈,해당없음,2024-07-03
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",19850311001,20230728,P123-207020300-1205,23,즉석식품류,23207,국/탕류,...,NaN,NaN,NaN,NaN,250ml(g),600g,1985031100113479,태원식품산업주,해당없음,2023-12-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",20080493074,20181114,P123-201020300-2999,23,즉석식품류,23201,밥류,...,NaN,NaN,NaN,NaN,210g,210g,20080493074492,농업회사법인주식회사한우물,해당없음,2021-06-30
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",20080493074,20181116,P123-201020300-3000,23,즉석식품류,23201,밥류,...,NaN,NaN,NaN,NaN,210g,210g,20080493074494,농업회사법인주식회사한우물,해당없음,2021-06-30
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",20080493074,20181109,P123-201020300-3001,23,즉석식품류,23201,밥류,...,NaN,NaN,NaN,NaN,210g,210g,20080493074490,농업회사법인주식회사한우물,해당없음,2021-06-30
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",20080493074,20181114,P123-201020300-3002,23,즉석식품류,23201,밥류,...,NaN,NaN,NaN,NaN,210g,210g,20080493074491,농업회사법인주식회사한우물,해당없음,2021-06-30


In [15]:
ingredient_df.columns

Index(['제품명', '업체명', '원재료명', '허가번호', '품목보고일자', '식품코드', '식품대분류코드', '식품대분류명',
       '대표식품코드', '대표식품명', '식품중분류코드', '식품중분류명', '식품소분류코드', '식품소분류명',
       '에너지(kcal)', '단백질(g)', '지방(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)',
       '철(mg)', '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민A(μg RAE)', '베타카로틴(μg)',
       '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민 C(mg)', '비타민 D(μg)',
       '콜레스테롤(mg)', '포화지방산(g)', '트랜스지방산(g)', '비오틴(μg)', '비타민 B6 / 피리독신(mg)',
       '비타민 B12(μg)', '엽산(μg DFE)', '판토텐산(mg)', '비타민 D3(μg)', '비타민 E(mg α-TE)',
       '비타민 K(μg)', '비타민 K1(μg)', '당알콜(g)', '알룰로오스(g)', '에리스리톨(g)',
       '불포화지방산(g)', 'EPA와 DHA의 합(mg)', '리놀레산(18:2(n-6))(g)',
       '알파 리놀렌산(18:3(n-3))(g)', '오메가3 지방산(g)', '오메가6 지방산(g)',
       '올레산(18:1(n-9))(mg)', '구리(μg)', '마그네슘(mg)', '망간(mg)', '몰리브덴(μg)',
       '셀레늄(μg)', '아연(mg)', '염소(mg)', '요오드(μg)', '크롬(μg)', '라이신(mg)', '류신(mg)',
       '메티오닌(mg)', '발린(mg)', '아르기닌(mg)', '이소류신(mg)', '타우린(mg)', '트레오닌(mg)',
       '트립토판(mg)', '페닐알라닌(mg)', '히스티딘(mg)', '1회 섭취참고량', 

In [16]:
ingredient_df['식품소분류코드'].unique()

array([ 10000001, 200000203, 230000203,  90000200,  90000302,  90000303,
        90000102, 140000201,  10000004,  30000204,  10000005, 230000202,
        80000003, 140000102, 130000204, 200000600,  70000302,  70000301,
        30000104,  10000002, 120000004, 130000202, 130000203,  70000107,
        70000105, 240000200,  80000002, 120000010, 160000302,  60000001,
       160000301,  70000117, 200000104, 130000101, 160000101,  80000001,
       160000701,  20000300, 160000600, 160000702, 160000703,  90000802,
        90000103,  40000700,  90000801,  30000205, 230000301, 100000400,
        90000700, 140000300, 230000201, 100000500,  70000101, 170000800,
        50000001, 140000101, 200000302, 200000400, 130000302,  80000004,
       120000013, 130000502,  60000004, 140000202,  70000103, 160000705,
        60000003,  90000401, 100000700, 200000301, 200000201,  90000502,
       190001400,  30000201, 160000400, 130000201, 160000704,  90000601,
        30000203, 110000102, 110000101, 190000204, 

In [17]:
ingredient_df.drop(['허가번호', '품목보고일자', '식품코드', '식품대분류코드',
       '대표식품코드', '대표식품명', '식품중분류코드', '식품중분류명', '식품소분류코드',
       '1회 섭취참고량', '식품중량', '품목제조보고번호','제조사명', '데이터생성일자'], axis=1, inplace=True)


ingredient_df


,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,메티오닌(mg),발린(mg),아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",과자류·빵류 또는 떡류,과자,395,0.00,0.50,88.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,300,10.00,15.00,30.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,75,2.00,4.00,7.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",수산가공식품류,액젓,134,10.50,0.10,23.20,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179,6.19,6.67,23.33,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186,4.76,7.14,25.71,2.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150,5.71,2.14,26.67,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186,5.71,6.19,26.19,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [18]:
ingredient_df.to_csv('refined_data(big_class).csv', index=False, encoding="utf-8-sig")

# 원재료 3개 이하는 제거

In [20]:
df = pd.read_csv('C:/Users/user/Desktop/gg/output_data/preprocessing_data/refined_data(big_class).csv', encoding = 'utf-8-sig')
df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,메티오닌(mg),발린(mg),아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",과자류·빵류 또는 떡류,과자,395,0.00,0.50,88.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,300,10.00,15.00,30.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,75,2.00,4.00,7.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",수산가공식품류,액젓,134,10.50,0.10,23.20,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179,6.19,6.67,23.33,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186,4.76,7.14,25.71,2.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150,5.71,2.14,26.67,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186,5.71,6.19,26.19,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [21]:
df = df[df['원재료명'].notnull() & (df['원재료명'].str.count(',') > 3)].reset_index(drop=True) # 원재료가 3개 이하는 제거

df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,메티오닌(mg),발린(mg),아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명
0,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
1,헴프씨드연꽃밥,따비온,"쌀, 찹쌀, 흑미찹쌀, 검정콩, 강낭콩, 건대추, 연근, 은행나무견과, 호박씨앗, ...",즉석식품류,즉석조리식품,156,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
2,헴프씨드연잎밥,따비온,"연잎, 쌀, 찹쌀, 흑미찹쌀, 강낭콩, 검정콩, 건대추, 연근, 은행나무견과, 호박...",즉석식품류,즉석조리식품,156,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
3,d국산동지팥죽,농업회사법인주강화드림,"쌀, 팥, 육수, 찹쌀, 천일염, 설탕",즉석식품류,즉석조리식품,78,3.00,0.00,16.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
4,국내산홍가리비초무침,라임에프앤비주,"깐양파, 가리비, 고추장, 미나리, 들깻잎, 당근, 홍고추, 식초, 양파청, 매실청...",절임류 또는 조림류,절임식품,75,6.67,1.00,10.83,5.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61385,행복한한상나주식곰탕국밥,농업회사법인주식회사한우물,"두류가공품, 쌀, 소고기, 대파, 채종유(유채유또는카놀라유), 알가열제품, 건면, ...",즉석식품류,즉석조리식품,205,5.71,6.19,31.43,2.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
61386,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179,6.19,6.67,23.33,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
61387,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186,4.76,7.14,25.71,2.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
61388,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150,5.71,2.14,26.67,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [25]:
# 술 데이터 살리기
df2 = pd.read_csv('./cleaned_food_data_merged.csv', encoding = 'utf-8-sig')
df2

C:\Users\user\AppData\Local\Temp\ipykernel_17252\379866350.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./cleaned_food_data_merged.csv', encoding = 'utf-8-sig')


,제품명,업체명,원재료명,허가번호,품목보고일자
0,NHY 구운어묵,정광푸드,"냉동연육, 전분, 밀가루, 정제염, 소르빈산칼륨, 글루코노-δ-락톤, L-글루타민산...",19900458016,20060110
1,VASAK 연근•참깨 김부각,(주)춘향골식품,"파래김, 찹쌀, 참깨, 콩기름(대두유), 천일염, 연근, 새우분말, 귀리분말, 다시...",20210496250,20241112
2,가시오가피 냉면,주식회사 금성에프앤비,"밀가루, 정제수, 타피오카전분, 가시오갈피분말, 알카리제, 정제염",19970401055,20020409
3,느루약주,구즉주조장,"정제수, 쌀, 국, 효모, 아스파탐, 아세설팜칼륨",20180012410,20211005
4,돌게장,(주)청호식품,"계피, 황기, 대추, 생강, 마늘, 간장, 돌게",20040294614,20040818
...,...,...,...,...,...
482261,산딸기청,서가,"산딸기, 비타민C, 자몽종자추출물, 올리고당, 레몬농축액, 당류가공품",20170538583,20220803
482262,산딸기치즈케익,주식회사 아델라7,"설탕, 산딸기잼향, 우유, 가공치즈, 빵류, 생크림, 버터, 레몬, 정제수, 계란노...",20200137067,20200629
482263,산딸기케이크,주식회사 레헴(삼송지점),"베이킹파우더, 기타가공품, 식품첨가물혼합제제, 당류가공품, 일반증류주, 과채주스, ...",20200335089,20200116
482264,산딸기케이크,주식회사경남제과,"전란, 밀가루, 백설탕, 식품첨가물혼합제제, 생크림, 콩기름(대두유), 마가린, 청...",20010247810,20220112


In [26]:
keywords = ['참이슬', '막걸리', '맥주', '새로', '진로']

# 키워드 중 하나라도 포함된 제품명 필터링
mask = df2['제품명'].str.contains('|'.join(keywords), case=False, na=False)

filtered_df = df2[mask].reset_index(drop=True)

In [27]:
filtered_df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,자기야 생 막걸리,농업회사 법인 주식회사 충주막걸리,"아세설팜칼륨, 입국, 쌀, 올리고당, 효모, 아스파탐, 사카린나트륨, 정제효소제, 정제수",20172880142,20200317
1,홈 막걸리12도,농업회사법인황칠본가(주),"입국, 멥쌀, 찹쌀, 효모, 누룩, 황칠나무추출물, 정제수",20220019116,20241205
2,홈 막걸리9도,농업회사법인황칠본가(주),"입국, 멥쌀, 찹쌀, 효모, 누룩, 황칠나무추출물, 정제수",20220019116,20241202
3,(땅콩)생막걸리,영동심천탁주공동제조장,"누룩, 과당, 밀가루, 구연산, 정제효소제, 아세설팜칼륨, 정제수, 땅콩 또는 견과...",20130020202,20180405
4,1.5도맥주,세븐브로이맥주 주식회사,"정제수, 맥아, 이산화탄소, 효모, 호프펠렛, 혼합제제, 향료, 셀룰라아제, 효모추출물",20110015205,20240404
...,...,...,...,...,...
2490,사임당생막걸리,사임당,"정제수, 팽화미, 입국, 물엿, 혼합제제, 정제효소제, 아세설팜칼륨",20172780104,20170123
2491,사화막걸리,농업회사법인 우포의아침 주식회사,"정제수, 찹쌀, 쌀, 입국, 누룩, 효모, 정제효소제, α-아밀라아제(세균성)",20130016119,20240822
2492,산더덕 생막걸리,농업회사법인아리랑주조주식회사,"정제수, 아스파탐, 삭카린나트륨, 올리고당, 효소, 효모, 더덕, 밀가루, 입국",20130020085,20160106
2493,산동생막걸리,산동양조장,"아스파탐, 효모, 정제효소제, 밀가루, 입국, 쌀, 정제수",20130018030,20240813


In [43]:
filtered_df.to_csv('alcohol.csv', encoding='utf-8-sig', index=False)

In [28]:
alcohol_df = pd.read_csv('alcohol.csv', encoding='utf-8-sig')
alcohol_df

,제품명,업체명,원재료명,허가번호,품목보고일자
0,자기야 생 막걸리,농업회사 법인 주식회사 충주막걸리,"아세설팜칼륨, 입국, 쌀, 올리고당, 효모, 아스파탐, 사카린나트륨, 정제효소제, 정제수",20172880142,20200317
1,홈 막걸리12도,농업회사법인황칠본가(주),"입국, 멥쌀, 찹쌀, 효모, 누룩, 황칠나무추출물, 정제수",20220019116,20241205
2,홈 막걸리9도,농업회사법인황칠본가(주),"입국, 멥쌀, 찹쌀, 효모, 누룩, 황칠나무추출물, 정제수",20220019116,20241202
3,(땅콩)생막걸리,영동심천탁주공동제조장,"누룩, 과당, 밀가루, 구연산, 정제효소제, 아세설팜칼륨, 정제수, 땅콩 또는 견과...",20130020202,20180405
4,1.5도맥주,세븐브로이맥주 주식회사,"정제수, 맥아, 이산화탄소, 효모, 호프펠렛, 혼합제제, 향료, 셀룰라아제, 효모추출물",20110015205,20240404
...,...,...,...,...,...
2490,사임당생막걸리,사임당,"정제수, 팽화미, 입국, 물엿, 혼합제제, 정제효소제, 아세설팜칼륨",20172780104,20170123
2491,사화막걸리,농업회사법인 우포의아침 주식회사,"정제수, 찹쌀, 쌀, 입국, 누룩, 효모, 정제효소제, α-아밀라아제(세균성)",20130016119,20240822
2492,산더덕 생막걸리,농업회사법인아리랑주조주식회사,"정제수, 아스파탐, 삭카린나트륨, 올리고당, 효소, 효모, 더덕, 밀가루, 입국",20130020085,20160106
2493,산동생막걸리,산동양조장,"아스파탐, 효모, 정제효소제, 밀가루, 입국, 쌀, 정제수",20130018030,20240813


In [29]:
merged_df = pd.merge(ingredient_df, alcohol_df, on='제품명', how='outer')

ingredient_df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,메티오닌(mg),발린(mg),아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",과자류·빵류 또는 떡류,과자,395,0.00,0.50,88.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,300,10.00,15.00,30.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,75,2.00,4.00,7.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",수산가공식품류,액젓,134,10.50,0.10,23.20,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81532,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179,6.19,6.67,23.33,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81533,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186,4.76,7.14,25.71,2.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81534,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150,5.71,2.14,26.67,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
81535,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186,5.71,6.19,26.19,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [49]:
ingredient_df.to_csv('test.csv', encoding='utf-8-sig', index=False)

In [30]:
combined_df = pd.concat([ingredient_df, alcohol_df], ignore_index=True, sort=False)

In [31]:
combined_df.to_csv('test02(big_class).csv', encoding='utf-8-sig', index=False)

In [32]:
test_df = pd.read_csv('test02(big_class).csv', encoding = 'utf-8-sig')
test_df

C:\Users\user\AppData\Local\Temp\ipykernel_17252\3605741088.py:1: DtypeWarning: Columns (3,4,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test02(big_class).csv', encoding = 'utf-8-sig')


,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명,허가번호,품목보고일자
0,팝콘,쌀로만식품,"옥수수, 백설탕, 탄산수소나트륨",과자류·빵류 또는 떡류,과자,395.0,0.0,0.5,88.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
1,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,300.0,10.0,15.0,30.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
2,페스츄리호두과자오리지널,주더베이커리랩제2공장,"조림류, 호두, 빵류",과자류·빵류 또는 떡류,과자,75.0,2.0,4.0,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
3,痲마한맛초피액젓,감포전촌젓갈,"초피나무열매, 액젓",수산가공식품류,액젓,134.0,10.5,0.1,23.2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
4,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335.0,32.0,15.0,18.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84027,사임당생막걸리,사임당,"정제수, 팽화미, 입국, 물엿, 혼합제제, 정제효소제, 아세설팜칼륨",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.017278e+10,20170123
84028,사화막걸리,농업회사법인 우포의아침 주식회사,"정제수, 찹쌀, 쌀, 입국, 누룩, 효모, 정제효소제, α-아밀라아제(세균성)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20240822
84029,산더덕 생막걸리,농업회사법인아리랑주조주식회사,"정제수, 아스파탐, 삭카린나트륨, 올리고당, 효소, 효모, 더덕, 밀가루, 입국",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20160106
84030,산동생막걸리,산동양조장,"아스파탐, 효모, 정제효소제, 밀가루, 입국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20240813


In [33]:
test_df = test_df[test_df['원재료명'].notnull() & (test_df['원재료명'].str.count(',') > 3)].reset_index(drop=True) # 원재료가 3개 이하는 제거

test_df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,아르기닌(mg),이소류신(mg),타우린(mg),트레오닌(mg),트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명,허가번호,품목보고일자
0,청기와타운양곰탕,태원식품산업주,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,...",즉석식품류,즉석조리식품,335.0,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
1,헴프씨드연꽃밥,따비온,"쌀, 찹쌀, 흑미찹쌀, 검정콩, 강낭콩, 건대추, 연근, 은행나무견과, 호박씨앗, ...",즉석식품류,즉석조리식품,156.0,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
2,헴프씨드연잎밥,따비온,"연잎, 쌀, 찹쌀, 흑미찹쌀, 강낭콩, 검정콩, 건대추, 연근, 은행나무견과, 호박...",즉석식품류,즉석조리식품,156.0,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
3,d국산동지팥죽,농업회사법인주강화드림,"쌀, 팥, 육수, 찹쌀, 천일염, 설탕",즉석식품류,즉석조리식품,78.0,3.00,0.00,16.00,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
4,국내산홍가리비초무침,라임에프앤비주,"깐양파, 가리비, 고추장, 미나리, 들깻잎, 당근, 홍고추, 식초, 양파청, 매실청...",절임류 또는 조림류,절임식품,75.0,6.67,1.00,10.83,5.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63880,사임당생막걸리,사임당,"정제수, 팽화미, 입국, 물엿, 혼합제제, 정제효소제, 아세설팜칼륨",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.017278e+10,20170123
63881,사화막걸리,농업회사법인 우포의아침 주식회사,"정제수, 찹쌀, 쌀, 입국, 누룩, 효모, 정제효소제, α-아밀라아제(세균성)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20240822
63882,산더덕 생막걸리,농업회사법인아리랑주조주식회사,"정제수, 아스파탐, 삭카린나트륨, 올리고당, 효소, 효모, 더덕, 밀가루, 입국",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20160106
63883,산동생막걸리,산동양조장,"아스파탐, 효모, 정제효소제, 밀가루, 입국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013002e+10,20240813


In [34]:
test_df.to_csv('refined_data_include_alcohol(big_class).csv', encoding='utf-8-sig', index=False)

In [43]:
label_df = pd.read_csv("C:/Users/user/Desktop/zerosweet/output_data/ingrediant_label(Ver 2.0).csv", encoding='cp949')

label_df

,원재료명,분류,혈당지수(GI),혈당영향,안전성
0,정제수,NaN,NaN,NaN,NaN
1,설탕,정제당,68,주의,과량 섭취 시 혈당 스파이크 주의
2,소스,NaN,NaN,NaN,NaN
3,정제소금,NaN,NaN,NaN,NaN
4,기타가공품,NaN,NaN,NaN,NaN
...,...,...,...,...,...
8938,비빔오징어젓,NaN,NaN,NaN,NaN
8939,조미분말믹스,NaN,NaN,NaN,NaN
8940,염장낙지,NaN,NaN,NaN,NaN
8941,NaN,NaN,NaN,NaN,NaN


In [44]:
label_df.dropna(how='all', axis=0)

,원재료명,분류,혈당지수(GI),혈당영향,안전성
0,정제수,NaN,NaN,NaN,NaN
1,설탕,정제당,68,주의,과량 섭취 시 혈당 스파이크 주의
2,소스,NaN,NaN,NaN,NaN
3,정제소금,NaN,NaN,NaN,NaN
4,기타가공품,NaN,NaN,NaN,NaN
...,...,...,...,...,...
8936,게장,NaN,NaN,NaN,NaN
8937,고구마플레이크,NaN,NaN,NaN,NaN
8938,비빔오징어젓,NaN,NaN,NaN,NaN
8939,조미분말믹스,NaN,NaN,NaN,NaN


In [45]:
label_df

,원재료명,분류,혈당지수(GI),혈당영향,안전성
0,정제수,NaN,NaN,NaN,NaN
1,설탕,정제당,68,주의,과량 섭취 시 혈당 스파이크 주의
2,소스,NaN,NaN,NaN,NaN
3,정제소금,NaN,NaN,NaN,NaN
4,기타가공품,NaN,NaN,NaN,NaN
...,...,...,...,...,...
8938,비빔오징어젓,NaN,NaN,NaN,NaN
8939,조미분말믹스,NaN,NaN,NaN,NaN
8940,염장낙지,NaN,NaN,NaN,NaN
8941,NaN,NaN,NaN,NaN,NaN


In [46]:
non_empty_gi_count = label_df['혈당지수(GI)'].notna().sum()
print(f"혈당지수(GI) 값이 존재하는 성분 수: {non_empty_gi_count}")

혈당지수(GI) 값이 존재하는 성분 수: 288


In [47]:
label_warning_index = label_df['혈당영향'] == '주의'

In [48]:
label_warning_index = label_df['혈당영향'] == '주의'
warning_label = label_df[label_warning_index]['원재료명'].to_list()
warning_label

['설탕',
 '당류가공품',
 '물엿',
 '백설탕',
 '기타과당',
 '포도당',
 '기타설탕',
 '갈색설탕',
 '슈가파우더',
 '액상과당',
 '분말결정포도당',
 '쌀조청',
 '결정과당',
 '기타엿',
 '트레할로스',
 '과당',
 '당시럽류',
 '사탕수수가루',
 '말토덱스트린혼합제제',
 '맥아추출액',
 '맥아엑기스',
 '당시럽',
 '맥아',
 '흑설탕향',
 '엿류',
 '말토덱스트린',
 '설탕시럽',
 '딸기잼시럽',
 '맥아호밀가루',
 '맥아분말',
 '맥아추출물분말',
 '사탕수수추출물',
 '당류',
 '고과당',
 '정제포도당',
 '전화당시럽',
 '함수결정포도당',
 '흑설탕',
 '바닐라시럽',
 '이온물엿',
 '맥아물엿',
 '엿기름당화액',
 '과당옥수수시럽',
 '찹쌀조청',
 '무수결정포도당',
 '당밀시럽',
 '현미시럽',
 '포도당시럽',
 '딸기시럽',
 '사탕수수시럽',
 '건조사탕수수분말',
 '조청',
 '물엿3',
 '황백당',
 '망고시럽',
 '보리맥아',
 '밀맥아분말',
 '레몬시럽',
 '밀시럽',
 '건조사탕수수',
 '슈가분말',
 '생강시럽농축액',
 '바닐라슈가',
 '함수포도당',
 '액상포도당',
 '슈가시럽',
 '맥아농축액',
 '사탕수수줄기',
 '사과시럽',
 '옥수수시럽',
 '요리당',
 '쌀엿',
 '사탕수수',
 '펄슈가',
 '포도시럽',
 '바나나시럽',
 '유기설탕',
 '무수포도당',
 '옥수수시럽농축액',
 '고과당옥수수시럽',
 '밀당화물',
 '커피시럽',
 '황설탕',
 '가루엿',
 '트레이할로스',
 '딸기향시럽',
 '카카오시럽',
 '결정(분말)포도당',
 '유기농황설탕',
 '블루베리시럽',
 '맥아조청',
 '오렌지시럽',
 '말토덱스트린분말',
 '체리시럽',
 '유기쌀조청',
 '이온엿',
 '우유시럽',
 '분말설탕',
 '배시럽',
 '카라멜시럽M',
 '액상카라멜',
 '바닐라맛시럽',
 '물엿82',
 '맥아추출고형분'

In [49]:
label_safe_index = label_df['혈당영향'] == '안전'
safe_label = label_df[label_safe_index]['원재료명'].to_list()
safe_label

['효소처리스테비아',
 '수크랄로스',
 '프락토올리고당',
 '올리고당',
 '아세설팜칼륨',
 '에리스리톨',
 '난소화성말토덱스트린(고시형)',
 'D-소비톨액',
 '알룰로오스',
 '자일리톨',
 'D-자일로오스',
 'D-소비톨',
 '스테비올배당체',
 '폴리덱스트로스',
 '이소말토올리고당',
 '아스파탐',
 '사카린나트륨',
 '이소말트',
 '사카린나트륨제제',
 '프락토올리고당(고시형)',
 '감초',
 '나한과추출물분말',
 '난소화성말토덱스트린분말(고시형)',
 '스테비텐후레쉬',
 '감초건조',
 '스테비아추출물',
 '감초농축액',
 '감초추출물',
 '자일리톨혼합제제',
 '올리고당가공품',
 'D-소르비톨액',
 '갈락토올리고당',
 'D-소르비톨',
 '말토올리고당',
 '락티톨',
 '삭카린나트륨제제',
 '삭카린나트륨',
 '스테비안45에스',
 'D-소르비톨혼합제제',
 '감초뿌리',
 '기타올리고당',
 '감초분말',
 '스테비아잎',
 '나한과분말',
 '만니톨',
 'STEVIOSIDE 스테비오사이드',
 '감초추출물분말',
 '감초농축분말',
 '나한과농축액',
 '자일로올리고당',
 '효소처리스테비아분말',
 '스테비오사이드혼합분말',
 '감초엑기스',
 'D-소르비톨액농축액',
 'D-소르비톨결정성 분말',
 '감초추출액',
 '감초엑기스분말',
 '나한과추출액',
 '감초뿌리줄기',
 '감초물',
 '이소말토올리고당(기능성원료인정제2005-10호)',
 '나한과추출물 혼합분말',
 '스테비온사이드',
 '스테비아',
 '효소처리 스테비아',
 '플락토올리고당',
 '감초추출물(신규원료)',
 '효소처리스테비아(원료변경)',
 '에리쓰리톨',
 '폴리덱스트로스(고시형)',
 '자일로오스',
 '수크로오스',
 '유기쌀올리고당',
 '자일리톨(기능성원료인정제2004-2호)',
 '볶은감초',
 '효모처리스테비아',
 '스테비아잎농축액',
 '스테비올배당체혼합제제',
 '감초뿌리박',
 '이소말토올리고당(기능성원

In [50]:
label_care_index = label_df['혈당영향'] == '유의'
care_label = label_df[label_care_index]['원재료명'].to_list()
care_label

['사양벌꿀',
 '벌꿀',
 '말티톨시럽',
 '폴리글리시톨시럽',
 'D-말티톨',
 '단풍당시럽',
 '용설란시럽',
 '레몬농축과즙',
 '배농축과즙',
 '벌꿀분말',
 '오렌지농축과즙',
 '벌꿀류',
 '포도농축과즙',
 '딸기농축과즙',
 '백포도농축과즙',
 '파인애플농축과즙',
 '메이플시럽향',
 '감귤농축과즙액',
 '메이플시럽분말',
 '골든시럽',
 '저당',
 '코코넛시럽',
 '벌꿀시럽',
 '아가베시럽',
 '신양벚나무열매농축과즙',
 '배청징농축과즙',
 '자몽농축과즙',
 '사과농축과즙',
 '청포도농축과즙',
 '당밀추출물',
 '잡화꿀',
 '바나나농축과즙',
 '키토올리고당(고시형)',
 '유자농축과즙액',
 '메이플향',
 '석류농축과즙',
 '블루베리농축과즙',
 'Red grape농축과즙',
 '개버즘단풍나무수액',
 '복숭아농축과즙',
 '매실농축과즙',
 '자두농축과즙',
 '멜론농축과즙액',
 '푸룬농축과즙',
 '말티톨액',
 '고추장당화물',
 '라즈베리농축과즙',
 '적포도농축과즙',
 '폴리글리시톨액',
 '당밀',
 '유기농레몬농축과즙',
 '메이플크리스피',
 '복숭아농축과즙액',
 '메이플시럽',
 '경단풍나무수액',
 '설탕단풍수액',
 '시트러스믹스농축과즙',
 '건조당밀',
 '아카시아꿀',
 '키토올리고당분말(고시형)',
 '벌꿀추출물',
 '유기농포도농축과즙',
 '유기농백포도농축과즙',
 '폴리글리시톨',
 '아카시아벌꿀',
 '식용당밀',
 '모과농축과즙액',
 '벌집꿀',
 '열대과일농축과즙',
 '키위농축과즙',
 '망고농축과즙',
 '벌꿀유자',
 '키위농축과즙액']

In [ ]:
import pandas as pd

# 1. 데이터 로딩
refined_df = pd.read_csv('refined_data_include_alcohol(big_class).csv', encoding='utf-8-sig')
ingredients_info_df = pd.read_csv('ingredients_info.csv', encoding='utf-8-sig')

# 2. 정보 매핑용 딕셔너리
info_dict = ingredients_info_df.set_index('원재료명').to_dict(orient='index')

# 3. 카테고리 + 정보 생성 함수
def categorize_with_info(ingredients):
    parts = [p.strip() for p in str(ingredients).split(',')]
    
    # 기본 카테고리 분류
    categorized = {
        '안전': [],
        '유의': [],
        '주의': [],
        '그외': [],
        '안전_정보': [],
        '유의_정보': [],
        '주의_정보': [],
        '그외_정보': []
    }

    for p in parts:
        info = info_dict.get(p)
        label = info['안전성'] if info else '그외'

        # 기본 분류
        categorized[label].append(p)

        # 정보가 있으면 정보 문자열 추가
        if info:
            info_str = f"{p}(GI:{info['혈당지수(GI)']}, 안전성:{info['안전성']})"
        else:
            info_str = p  # 정보 없으면 그대로

        categorized[f'{label}_정보'].append(info_str)

    return pd.Series({
        '안전': ', '.join(categorized['안전']),
        '유의': ', '.join(categorized['유의']),
        '주의': ', '.join(categorized['주의']),
        '그외': ', '.join(categorized['그외']),
        '안전_정보': ', '.join(categorized['안전_정보']),
        '유의_정보': ', '.join(categorized['유의_정보']),
        '주의_정보': ', '.join(categorized['주의_정보']),
        '그외_정보': ', '.join(categorized['그외_정보']),
    })

# 4. 적용
refined_df = refined_df.join(refined_df['원재료명'].apply(categorize_with_info))

# 5. 저장
refined_df.to_csv('refined_data_with_info_columns.csv', index=False, encoding='utf-8-sig')

In [51]:
refined_df = pd.read_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_with_gi_non_duplicate_full_category(big_class).csv', encoding='utf-8-sig')

refined_df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,유통업체명,허가번호,품목보고일자,안전,유의,주의,그외,당지수(GI),안전성,인증
0,열수한강막걸리다크15.5,농업회사법인 봄내양조장주식회사,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,0
1,열수한강막걸리연13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,0
2,열수한강막걸리연15.5,농업회사법인 봄내양조장주식회사,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,0
3,열수한강막걸리오13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,0
4,열수한강막걸리오15.5,농업회사법인 봄내양조장주식회사,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54031,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179.0,6.19,6.67,23.33,0.95,...,해당없음,NaN,NaN,NaN,NaN,NaN,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",NaN,NaN,0
54032,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186.0,4.76,7.14,25.71,2.86,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",설탕:65,설탕:과량 섭취 시 혈당 스파이크 주의,0
54033,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150.0,5.71,2.14,26.67,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",설탕:65,설탕:과량 섭취 시 혈당 스파이크 주의,0
54034,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186.0,5.71,6.19,26.19,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",설탕:65,설탕:과량 섭취 시 혈당 스파이크 주의,0


In [52]:
refined_df.columns

Index(['제품명', '업체명', '원재료명', '식품대분류명', '식품소분류명', '에너지(kcal)', '단백질(g)',
       '지방(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)', '인(mg)',
       '칼륨(mg)', '나트륨(mg)', '비타민A(μg RAE)', '베타카로틴(μg)', '티아민(mg)',
       '리보플라빈(mg)', '니아신(mg)', '비타민 C(mg)', '비타민 D(μg)', '콜레스테롤(mg)',
       '포화지방산(g)', '트랜스지방산(g)', '비오틴(μg)', '비타민 B6 / 피리독신(mg)', '비타민 B12(μg)',
       '엽산(μg DFE)', '판토텐산(mg)', '비타민 D3(μg)', '비타민 E(mg α-TE)', '비타민 K(μg)',
       '비타민 K1(μg)', '당알콜(g)', '알룰로오스(g)', '에리스리톨(g)', '불포화지방산(g)',
       'EPA와 DHA의 합(mg)', '리놀레산(18:2(n-6))(g)', '알파 리놀렌산(18:3(n-3))(g)',
       '오메가3 지방산(g)', '오메가6 지방산(g)', '올레산(18:1(n-9))(mg)', '구리(μg)',
       '마그네슘(mg)', '망간(mg)', '몰리브덴(μg)', '셀레늄(μg)', '아연(mg)', '염소(mg)',
       '요오드(μg)', '크롬(μg)', '라이신(mg)', '류신(mg)', '메티오닌(mg)', '발린(mg)',
       '아르기닌(mg)', '이소류신(mg)', '타우린(mg)', '트레오닌(mg)', '트립토판(mg)', '페닐알라닌(mg)',
       '히스티딘(mg)', '유통업체명', '허가번호', '품목보고일자', '안전', '유의', '주의', '그외',
       '당지수(GI)', '안전성', '인증'],
      dtype='object')

In [53]:
# 예시: 컬럼 이름 5개
columns_to_clear = ['당지수(GI)', '안전', '유의', '주의', '그외', '인증', '안전성']

# 해당 컬럼들의 값을 빈 문자열로 초기화
refined_df[columns_to_clear] = ""

In [ ]:
# refined_df = pd.read_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_with_gi_non_duplicate(big_class).csv', encoding='utf-8-sig')

# 원재료명을 분리해서 각 카테고리별로 다시 넣어주는 함수
def categorize_ingredients(ingredients):
    parts = [p.strip() for p in str(ingredients).split(',') if p.strip()]
    safe, care, warning, nothing = [], [], [], []

    for p in parts:
        if any(k in p for k in safe_label):
            safe.append(p)
        elif any(k in p for k in care_label):
            care.append(p)
        elif any(k in p for k in warning_label):
            warning.append(p)
        else:
            nothing.append(p)

    return pd.Series({
        '안전': ', '.join(safe),
        '유의': ', '.join(care),
        '주의': ', '.join(warning),
        '그외': ', '.join(nothing)
    })

# 적용해서 새 컬럼 생성
refined_df[['안전', '유의', '주의', '그외']] = refined_df['원재료명'].apply(categorize_ingredients)

# 결과 확인
print(refined_df[['안전', '유의', '주의', '그외']].head())

# 필요하면 다시 CSV로 저장
refined_df.to_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_new_label(big_class).csv',index=False, encoding='utf-8-sig')

  안전 유의 주의                                     그외
0            커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수
1              율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제
2              연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수
3           율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제
4           오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수


In [45]:
def remove_categorized_ingredients(row):
    # 4개 카테고리의 원재료 문자열 합치기 → 리스트로 변환
    all_categorized = ','.join([row['안전'], row['유의'], row['주의'], row['그외']])
    categorized_list = [item.strip() for item in all_categorized.split(',') if item.strip()]
    
    # 원재료명에서 분리된 원재료 목록
    original_parts = [item.strip() for item in str(row['원재료명']).split(',')]
    
    # categorized에 포함되지 않은 것만 남기기
    remaining = [item for item in original_parts if item not in categorized_list]
    
    return ', '.join(remaining)

# 원재료명 업데이트
refined_df['원재료명'] = refined_df.apply(remove_categorized_ingredients, axis=1)

In [46]:
refined_df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,트립토판(mg),페닐알라닌(mg),히스티딘(mg),유통업체명,허가번호,품목보고일자,안전,유의,주의,그외
0,청기와타운양곰탕,태원식품산업주,,즉석식품류,즉석조리식품,335.0,32.00,15.00,18.00,5.00,...,NaN,NaN,NaN,해당없음,NaN,NaN,,,,"정제수, 양념육, 양념육, 소고기, 대파, 식육함유가공품, 고춧가루, 소스, 소스,..."
1,헴프씨드연꽃밥,따비온,,즉석식품류,즉석조리식품,156.0,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,해당없음,NaN,NaN,,,,"쌀, 찹쌀, 흑미찹쌀, 검정콩, 강낭콩, 건대추, 연근, 은행나무견과, 호박씨앗, ..."
2,헴프씨드연잎밥,따비온,,즉석식품류,즉석조리식품,156.0,3.87,0.94,33.09,1.26,...,NaN,NaN,NaN,해당없음,NaN,NaN,,,,"연잎, 쌀, 찹쌀, 흑미찹쌀, 강낭콩, 검정콩, 건대추, 연근, 은행나무견과, 호박..."
3,d국산동지팥죽,농업회사법인주강화드림,,즉석식품류,즉석조리식품,78.0,3.00,0.00,16.00,5.00,...,NaN,NaN,NaN,해당없음,NaN,NaN,,,설탕,"쌀, 팥, 육수, 찹쌀, 천일염"
4,국내산홍가리비초무침,라임에프앤비주,,절임류 또는 조림류,절임식품,75.0,6.67,1.00,10.83,5.83,...,NaN,NaN,NaN,해당없음,NaN,NaN,,,백설탕,"깐양파, 가리비, 고추장, 미나리, 들깻잎, 당근, 홍고추, 식초, 양파청, 매실청..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63880,사임당생막걸리,사임당,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.017278e+10,20170123,아세설팜칼륨,,물엿,"정제수, 팽화미, 입국, 혼합제제, 정제효소제"
63881,사화막걸리,농업회사법인 우포의아침 주식회사,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.013002e+10,20240822,,,,"정제수, 찹쌀, 쌀, 입국, 누룩, 효모, 정제효소제, α-아밀라아제(세균성)"
63882,산더덕 생막걸리,농업회사법인아리랑주조주식회사,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.013002e+10,20160106,"아스파탐, 삭카린나트륨",올리고당,,"정제수, 효소, 효모, 더덕, 밀가루, 입국"
63883,산동생막걸리,산동양조장,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.013002e+10,20240813,아스파탐,,,"효모, 정제효소제, 밀가루, 입국, 쌀, 정제수"


In [ ]:
refined_df.to_csv('refined_data_include_alcohol(big_class).csv',index=False, encoding='utf-8-sig')

In [56]:
import pandas as pd
# refined_df = pd.read_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_non_duplicate_full_category(big_class).csv', encoding='utf-8-sig')

# refined_df : 제품 정보
# label_df : 성분 정보 ('원재료명', '당지수(GI)', '안전성' 컬럼 있음)

# 딕셔너리로 매핑
gi_dict = label_df.set_index('원재료명')['혈당지수(GI)'].to_dict()
safety_dict = label_df.set_index('원재료명')['안전성'].to_dict()

# 정보 추출 함수
def extract_info(ingredient_str, info_dict):
    ingredients = [i.strip() for i in str(ingredient_str).split(',') if i.strip()]
    result = []
    for i in ingredients:
        value = info_dict.get(i)
        if pd.notna(value):
            result.append(f"{i}:{value}")
    return ', '.join(result)

# 전체 성분 문자열: 모든 카테고리 합치기
def combine_all_ingredients(row):
    return ', '.join(filter(None, [
        str(row.get('안전', '')),
        str(row.get('유의', '')),
        str(row.get('주의', '')),
        str(row.get('그외', ''))
    ]))

# 통합 성분 컬럼 만들기
refined_df['전체성분'] = refined_df.apply(combine_all_ingredients, axis=1)

# 두 개의 새 컬럼 추가
refined_df['당지수(GI)'] = refined_df['전체성분'].apply(lambda x: extract_info(x, gi_dict))
refined_df['안전성'] = refined_df['전체성분'].apply(lambda x: extract_info(x, safety_dict))

# 원하면 '전체성분'은 삭제해도 됨
refined_df.drop(columns=['전체성분'], inplace=True)

# 결과 확인
print(refined_df[['당지수(GI)', '안전성']].head())

# 저장 (선택)
refined_df.to_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_new_label(big_class).csv',index=False, encoding='utf-8-sig')

  당지수(GI) 안전성
0            
1            
2            
3            
4            


In [57]:
def is_blank_or_nan(s):
    return pd.isna(s) or str(s).strip() == ''

condition = (
    refined_df['안전'].notna() &
    refined_df['유의'].apply(is_blank_or_nan) &
    refined_df['주의'].apply(is_blank_or_nan)
)
refined_df['인증'] = np.where(condition, 1, 0)

In [58]:
refined_df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,유통업체명,허가번호,품목보고일자,안전,유의,주의,그외,당지수(GI),안전성,인증
0,열수한강막걸리다크15.5,농업회사법인 봄내양조장주식회사,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,,,,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",,,1
1,열수한강막걸리연13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,,,,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",,,1
2,열수한강막걸리연15.5,농업회사법인 봄내양조장주식회사,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,,,,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",,,1
3,열수한강막걸리오13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,,,,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",,,1
4,열수한강막걸리오15.5,농업회사법인 봄내양조장주식회사,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,,,,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54031,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179.0,6.19,6.67,23.33,0.95,...,해당없음,NaN,NaN,,,,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",,,1
54032,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186.0,4.76,7.14,25.71,2.86,...,해당없음,NaN,NaN,,,설탕,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54033,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150.0,5.71,2.14,26.67,0.95,...,해당없음,NaN,NaN,,,설탕,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54034,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186.0,5.71,6.19,26.19,0.95,...,해당없음,NaN,NaN,,,설탕,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0


In [ ]:
refined_df.to_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_new_label(big_class).csv',index=False, encoding='utf-8-sig')

# 데이터 채우기
 * 탄수화물, 단백질, 지방, 당류 성분 NaN 혹은 NULL, 빈 값 0으로 채우기 ( 전처리 과정에서 없어짐 )

In [8]:
import pandas as pd

df = pd.read_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_new_label(big_class).csv', encoding = 'utf-8-sig')
df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,유통업체명,허가번호,품목보고일자,안전,유의,주의,그외,당지수(GI),안전성,인증
0,열수한강막걸리다크15.5,농업회사법인 봄내양조장주식회사,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
1,열수한강막걸리연13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,1
2,열수한강막걸리연15.5,농업회사법인 봄내양조장주식회사,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
3,열수한강막걸리오13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,1
4,열수한강막걸리오15.5,농업회사법인 봄내양조장주식회사,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54031,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179.0,6.19,6.67,23.33,0.95,...,해당없음,NaN,NaN,NaN,NaN,NaN,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",NaN,NaN,1
54032,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186.0,4.76,7.14,25.71,2.86,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54033,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150.0,5.71,2.14,26.67,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54034,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186.0,5.71,6.19,26.19,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0


In [9]:
df.columns

Index(['제품명', '업체명', '원재료명', '식품대분류명', '식품소분류명', '에너지(kcal)', '단백질(g)',
       '지방(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)', '인(mg)',
       '칼륨(mg)', '나트륨(mg)', '비타민A(μg RAE)', '베타카로틴(μg)', '티아민(mg)',
       '리보플라빈(mg)', '니아신(mg)', '비타민 C(mg)', '비타민 D(μg)', '콜레스테롤(mg)',
       '포화지방산(g)', '트랜스지방산(g)', '비오틴(μg)', '비타민 B6 / 피리독신(mg)', '비타민 B12(μg)',
       '엽산(μg DFE)', '판토텐산(mg)', '비타민 D3(μg)', '비타민 E(mg α-TE)', '비타민 K(μg)',
       '비타민 K1(μg)', '당알콜(g)', '알룰로오스(g)', '에리스리톨(g)', '불포화지방산(g)',
       'EPA와 DHA의 합(mg)', '리놀레산(18:2(n-6))(g)', '알파 리놀렌산(18:3(n-3))(g)',
       '오메가3 지방산(g)', '오메가6 지방산(g)', '올레산(18:1(n-9))(mg)', '구리(μg)',
       '마그네슘(mg)', '망간(mg)', '몰리브덴(μg)', '셀레늄(μg)', '아연(mg)', '염소(mg)',
       '요오드(μg)', '크롬(μg)', '라이신(mg)', '류신(mg)', '메티오닌(mg)', '발린(mg)',
       '아르기닌(mg)', '이소류신(mg)', '타우린(mg)', '트레오닌(mg)', '트립토판(mg)', '페닐알라닌(mg)',
       '히스티딘(mg)', '유통업체명', '허가번호', '품목보고일자', '안전', '유의', '주의', '그외',
       '당지수(GI)', '안전성', '인증'],
      dtype='object')

In [20]:
essential_col = ['단백질(g)', '지방(g)', '탄수화물(g)', '당류(g)']

df[essential_col] = df[essential_col].fillna(0)
df

,제품명,업체명,원재료명,식품대분류명,식품소분류명,에너지(kcal),단백질(g),지방(g),탄수화물(g),당류(g),...,유통업체명,허가번호,품목보고일자,안전,유의,주의,그외,당지수(GI),안전성,인증
0,열수한강막걸리다크15.5,농업회사법인 봄내양조장주식회사,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"커피원두, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
1,열수한강막걸리연13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 연잎, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,1
2,열수한강막걸리연15.5,농업회사법인 봄내양조장주식회사,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"연잎, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
3,열수한강막걸리오13,농업회사법인 봄내양조장주식회사,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"율무쌀, 깐녹두, 오미자열매, 정제수, 쌀, 국, 효모, 정제효소제",NaN,NaN,1
4,열수한강막걸리오15.5,농업회사법인 봄내양조장주식회사,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,NaN,2.023000e+10,20250409.0,NaN,NaN,NaN,"오미자열매, 깐녹두, 율무쌀, 정제효소제, 효모, 국, 쌀, 정제수",NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54031,행복한한상소고기미역국밥,농업회사법인주식회사한우물,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",즉석식품류,즉석조리식품,179.0,6.19,6.67,23.33,0.95,...,해당없음,NaN,NaN,NaN,NaN,NaN,"기타가공품, 쌀, 소고기, 미역, 참기름, 소스, 채종유(유채유또는카놀라유), 마늘...",NaN,NaN,1
54032,행복한한상소고기육개장국밥,농업회사법인주식회사한우물,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",즉석식품류,즉석조리식품,186.0,4.76,7.14,25.71,2.86,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 대파, 소스, 소고기, 채종유(유채유또는카놀라유), 건면, 향미유, 마늘, 식...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54033,행복한한상시원한황태국밥,농업회사법인주식회사한우물,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",즉석식품류,즉석조리식품,150.0,5.71,2.14,26.67,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"대파, 무, 콩나물, 쌀, 소스, 황태채, 알가열제품, 채종유(유채유또는카놀라유),...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0
54034,행복한한상토종순대국밥,농업회사법인주식회사한우물,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",즉석식품류,즉석조리식품,186.0,5.71,6.19,26.19,0.95,...,해당없음,NaN,NaN,NaN,NaN,설탕,"쌀, 즉석조리식품, 양념육, 기타가공품, 대파, 부추, 청양고추, 채종유(유채유또는...",설탕:68,설탕:과량 섭취 시 혈당 스파이크 주의,0


In [22]:
df.to_csv('C:/Users/user/Desktop/zerosweet/output_data/preprocessing_data/Ver.05/refined_new_label(big_class).csv',index=False, encoding='utf-8-sig')